In [2]:
import re
import emoji
import random
import pandas as pd
from pandas import read_csv
from opencc import OpenCC
import os, itertools
from itertools import groupby
from datetime import datetime

# Organizing Facebook Content

In [4]:
### load data from Fb_crawling_output
df = read_csv('fb_crawling/data/fb_crawling_output.csv', encoding='utf8',index_col=0)

FileNotFoundError: [Errno 2] File b'fb_crawling/data/fb_crawling_output.csv' does not exist: b'fb_crawling/data/fb_crawling_output.csv'

In [3]:
### Remove non-"店家" post ###
temp_p_lst = []
temp_d_lst = []
temp_p = ''
temp_d = ''

for post_content, date_time in zip(df['Description'].astype(str), df['published'].astype(str)):
    #cc = OpenCC('s2tw') # 簡轉繁
    #post_content = cc.convert(post_content)
    post = post_content.replace(' ','').replace('￼￼','').replace('￼￼￼','').replace('\n','') \
        .replace('$','').replace('／','').replace('…','').replace('：',':').replace(':',':').replace('%','') \
        .replace(u'\u3000',u'').replace('▎','').replace('）',')').replace('（','(').replace('O','') \
        .replace('【',' ').replace('】',':').replace('■','').replace('◆','').replace('●','').replace('★','') \
        .replace('1.','').replace('2.','').replace('3.','').replace('4.','').replace('5.','') \
        .replace('6.','').replace('7.','') \
        .replace('\'','').replace('_','').replace('☆','') \
        .replace('▎','').replace('🁢',' ').replace('-','').replace('◎','')\
        .replace('更多','').replace('分隔線','').replace('NT','') \
        .replace('▎店家:','店家:').replace('▎店　　名:','店家:') \
        .replace('店家/鄰近','店家:').replace('店家/鄰近地標:','店家:').replace('店家鄰近地點:','店家:') \
        .replace('店家/鄰近地點:','店家:') \
        .replace('店名:','店家:').replace('▎店　　家:','店家:').replace(' 店家','店家:')\
        .replace('【店家】','店家:').replace('店家名稱:','店家:').replace('店家資訊:','店家:') \
        .replace('鄰近:','%鄰近地點:').replace('所在地區','%鄰近地點:') \
        .replace('鄰近位置:','%鄰近地點:').replace('地點:','%鄰近地點:') \
        .replace('所在地區/鄰近位置:','%鄰近地點:') \
        .replace('鄰近地點','%鄰近地點').replace('臨近地點:','%鄰近地點:').replace('地址:','%鄰近地點:') \
        .replace('鄰近地區','%鄰近地點').replace('區域','%鄰近地點:').replace('地區/鄰近','%鄰近地點:')\
        .replace('位置:','%鄰近地點:').replace('位置','%鄰近地點:')\
        .replace('拉麵名稱','%.G拉麵名稱').replace('餐點名稱:','%.G拉麵名稱:')\
        .replace('餐點:','%.G拉麵名稱:').replace('拉麵品項:','%.G拉麵名稱')\
        .replace('品項:','%.G拉麵名稱:').replace('品名:','%.G拉麵名稱:')\
        .replace('名稱:','%.G拉麵名稱:').replace('品項價格:','%.G拉麵名稱:')\
        .replace('配置:','Z配置').replace('配　　置','Z配置').replace('配置(','Z配置').replace('▎配置:','Z配置')\
        .replace('心得感想:','Z').replace('感想:','Z').replace('心得:','Z') \
        .replace('完食感受:','Z').replace('完食感想:','Z') \
        .replace('味增','味噌').replace('食八番','食8番').replace('五ノ','五之').replace('総','總')
    
    if "店家" in post[:5]:
        temp_p_lst.append(post)

        for word in date_time:
            temp_d += word
            if word == '日':
                break
        if ('小時' in temp_d) or ('分鐘' in temp_d):
            temp_d = '2020年12月2日'   # Change the current date as nedded
        elif ('昨天' in temp_d):
            temp_d = '2020年12月1日'   # Change the date as nedded
        elif '年' not in temp_d:
            temp_d = '2020年' + temp_d      
        temp_d_lst.append(temp_d)
        temp_d = ''

In [4]:
### Classifiy posts according to the standard format
unorganized_shops = []
unorganized_date = []
ramen_shop_raw = []
ramen_name_raw = []
ramen_review_raw = []
ramen_date_raw = []

for shops, date in zip(temp_p_lst, temp_d_lst):
    if ('%'not in shops or 'G' not in shops or 'Z' not in shops \
        or shops.index('Z')>shops.index('G')+80) :
        unorganized_shops.append(shops)
        unorganized_date.append(date)      
    else:
        ramen_shop_raw.append(shops[:shops.index('%')])
        ramen_name_raw.append(shops[shops.index('G')+1:shops.index('Z')])
        ramen_review_raw.append(shops[shops.index('Z')+1:shops.index('Z')+265]+'...')
        ramen_date_raw.append(date)

In [5]:
#### second filtering
unorganized_unorganized_shops = []
unorganized_unorganized_date = []

for shops, date in zip(unorganized_shops, unorganized_date):
    if ('G' in shops and '0' in shops):
        ramen_shop_raw.append(shops[:shops.index('%')])
        ramen_name_raw.append(shops[shops.index('G')+1 : shops.index('G')+35])
        ramen_review_raw.append(shops[shops.index('G')+15 : shops.index('G')+285]+'...')
        ramen_date_raw.append(date)
    else:
        unorganized_unorganized_shops.append(shops)
        unorganized_unorganized_date.append(date)

In [6]:
### Dealing with shop_name
ramen_shop_list = []

for shops in ramen_shop_raw:
    shops.replace('店家','').replace('#','').replace('鄰近','')
    if ':' in shops:
        shops = shops[shops.index(':')+1:]
    shops = emoji.demojize(shops)
    shops = shops.replace('／','')
    shops = re.sub(':\S+?:', ' ', shops)
    if ':' in shops:
        shops = shops[:shops.index(':')]
    if len(shops) <= 1:
        shops = ''
    if '地址' in shops and '用餐' in shops:
        if shops.index('地') < shops.index('用'):
            ramen_shop_list.append(shops[:shops.index('地')])
        else:
            ramen_shop_list.append(shops[:shops.index('用')])
    elif '地址' in shops:
        ramen_shop_list.append(shops[:shops.index('地')])
    elif '用餐' in shops:
        ramen_shop_list.append(shops[:shops.index('用')])
    else:
        ramen_shop_list.append(shops)
        
### Double check shop_name
ramen_shop_list_final = []
for shop in ramen_shop_list:
    shop = re.sub(r'[^\w\s]','',shop)
    shop = shop.replace('#','').replace('鄰近','').replace('臨近','').replace('排隊狀況','') \
    .replace('日期','').replace('店家','').replace('附近','').replace('《','').replace('》','') \
    .replace('。','').replace('、',' ').replace('｜','').replace('？','').replace('地區','') \
    .replace('(','').replace(')','').replace('·','').replace('/','').replace('地標','') \
    .replace('※','').replace('ぁ','あ').replace('²','2').replace(' ','')
    if len(shop) >= 31:
        shop = shop[:31]
    if '捷運' in shop:
        shop = shop[:shop.index('捷運')]
    ramen_shop_list_final.append(shop)  

In [7]:
### Dealing with ramen_name 
ramen_name_list = []
ramen_name_list_final = []

for names in ramen_name_raw:
    names = emoji.demojize(names)
    names = re.sub(':\S+?:', ' ', names)
    new_name = names.replace('拉麵%.G','').replace('%.G','').replace('#','').replace('鄰近','')
    last_ch = new_name[-1]
    first_ch = new_name[0]
    # ramen_name_list = []
    if ('0' in new_name) and ('00' not in new_name) and ('2020' not in new_name) \
        and (last_ch != ')'):
        if new_name[-1] == '元' or new_name[-2:] == '日元':
            ramen_name_list.append(new_name[:new_name.index('元')+1])
        elif new_name[-2:] == '日圓':
            ramen_name_list.append(new_name[:new_name.index('圓')+1])
        else:
            ramen_name_list.append(new_name[:new_name.index('0')+1])
    elif '00' in new_name and last_ch != ')' and '2020' not in new_name:
        ramen_name_list.append(new_name[:new_name.index('00')+2])
    elif first_ch != '拉' and '拉' in new_name:
        ramen_name_list.append(new_name[new_name.index('拉'):])
    elif '0' not in new_name and '00' not in new_name and '/' not in new_name\
        and last_ch.isdigit() == False and last_ch != '麵' \
        and last_ch != ')'and '+' not in new_name:
        new_point =  new_name.replace('麵','麵H').replace('拉麵H名稱','拉麵名稱')
        if 'H' in new_point:
            ramen_name_list.append(new_point[:new_point.index('H')])
        else:
            ramen_name_list.append(new_point)
    else:
        ramen_name_list.append(new_name)
        
### Double check ramen_name
for name in ramen_name_list:
    if ':' in name:
        name = name[name.index(':')+1:]
    name = name.replace('拉麵名稱/價格','').replace('拉麵名稱','').replace('拉麵名稱價格','').replace('價格','')
    ramen_name_list_final.append(name)  

In [8]:
### Dealing with fb_review
ramen_review_list = []
ramen_review_list_final = []

for name, reviews in zip(ramen_name_list, ramen_review_raw):
    new_reviews = reviews.replace('拉麵%.G','').replace('%.G','').replace('%','').replace('$','')
    first_few_words = new_reviews[0:6]
    if '配置'not in first_few_words:
        for i in range(-(len(name)),0,1):
            if (name[i:]) == (new_reviews[:-i]):
                updated_review = new_reviews[-i:]
                ramen_review_list.append(updated_review)
                break
            elif (i == -1) and (ramen_name_list[i:]) != (ramen_review_raw[:-i]):
                pattern="[\u4e00-\u9fa5]+" 
                regex = re.compile(pattern)
                results =  regex.findall(new_reviews)
                results_to_str =' '.join([str(elem) for elem in results]) 
                ramen_review_list.append(results_to_str)
    else:
        ramen_review_list.append(new_reviews)

### Double check fb_review
punc = [',' , '.' , '，', '。', ')',' ','……','；','、','\'','/','?','元']
for reviews in ramen_review_list:
    if reviews[0] in punc:
        reviews = reviews[1:]
    ramen_review_list_final.append(reviews)

In [9]:
### Remove emoji in fb_review
review_lst = []
def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ''.join([str for str in text if not any(i in str for i in emoji_list)])
    return clean_text

for r in ramen_review_list_final:
    r = give_emoji_free_text(r)
    review_lst.append(r)

In [10]:
### Create stem_store_name
stem_store = []

for store in ramen_shop_list_final:
    store = store.lower()
    store = re.sub(r'[^\w\s]','',store)
    store = store.replace('総','總').replace('麺','麵').replace('醤','醬').replace('らーめん七面','七面') \
                .replace('臺南白露','白露').replace('鷹流東京豚骨拉麵極匠','鷹流極匠') \
                .replace('太陽蕃茄EXPRESS','太陽蕃茄拉麵EXPRESS') \
                .replace('鷹流東京醬油拉麵蘭丸','鷹流蘭丸').replace('鷹流臺灣本店','鷹流拉麵臺灣本店') \
                .replace('ラーメン','拉麵').replace('らーめん','拉麵').replace('柑橘shin','柑橘shinn')  \
                .replace('麵屋壹之穴','麵屋壹之穴ichi').replace('の','之').replace('aqua2','') \
                .replace('麵鋪','麵舖').replace('台湾','臺灣').replace('番茄','蕃茄')  \
                .replace('台','臺').replace('麵や青鈴','麵屋青鈴')
    stem_store.append(store)

In [11]:
### Dealing with Datetime (string -> datetime)
ramen_date_final = []
for date in ramen_date_raw:
    date = date[2:]
    date = date.replace('年','/').replace('月','/').replace('日','')
    date_obj = datetime.strptime(date, '%y/%m/%d')
    ramen_date_final.append(date_obj)

In [12]:
#### Output
df_output = pd.DataFrame(list(zip(*[ramen_shop_list_final, stem_store, ramen_date_final, ramen_name_list_final, review_lst])))
col_names = ['stores', 'stem_store', 'create_on', 'ramen_name', 'fb_review']
df_output.columns = col_names
df_output["stores_len"] = df_output["stores"].str.len()
df_output["ramen_name_len"] = df_output["ramen_name"].str.len() 
df_output["fb_review_len"] = df_output["fb_review"].str.len() 
df_output_ = df_output[(df_output['stores_len'] > 1.0)]
df_output_ = df_output_[(df_output['ramen_name_len'] > 1.0)]
df_output_ = df_output_[(df_output['fb_review_len'] > 2.0)]

df_new = df_output_.sort_values(by=['stores', 'ramen_name_len'])
df_new = df_new.reset_index(drop=True)
df_new = df_new.drop(columns=['stores_len','ramen_name_len','fb_review_len'])
df_new = df_new.sort_index(axis=0 ,ascending=True)
df_new = df_new.iloc[::-1]
df_new = df_new.reset_index(drop=True)

/Users/yuchiacheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/Users/yuchiacheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


# Mapping store_name with Map (create store_id) -> TABLE3

In [13]:
stores_input = list(df_new['stem_store'])
shop_sorted = sorted(list(set(stores_input)))

In [14]:
### First GROUPING
util_func = lambda x: x[0]  
first_sort = [list(ele) for i, ele in groupby(shop_sorted, util_func)] 
lst = list(itertools.chain(*first_sort))

In [15]:
### SECOND GROUPING
unique = []
second_sort = []

util_func = lambda x: x[1]
for item in first_sort:
    if len(item) > 1:
        temp = sorted(item, key = util_func)
        second_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
    else:
        unique.append(item[0])

second_sort_len = []

for item_1 in second_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            second_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [16]:
### THIRD GROUPING
third_sort = []

util_func = lambda x: x[2]
for item in second_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' or item[0][:4] == '太陽' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵':
        third_sort.append([item])   
    else:
        if len(item) > 1 and len(item[0]) >= 3:
            temp = sorted(item, key = util_func)
            third_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
        elif len(item) > 1 and len(item[0]) == 2:
            third_sort.append([item])
        else:
            unique.append(item[0])
        
third_sort_len = []

for item_1 in third_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            third_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [17]:
### FOURTH GROUPING
fourth_sort = []

util_func = lambda x: x[3]
for item in third_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' or item[0][:4] == '太陽' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵':
        fourth_sort.append([item])   
    else:
        if len(item) > 1 and len(item[0]) >= 4:
            temp = sorted(item, key = util_func)
            fourth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 3:
            fourth_sort.append([item])
        else:
            unique.append(item[0])

fourth_sort_len = []

for item_1 in fourth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fourth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [18]:
### Double Check
done = []
unclassified_group = []

for item in fourth_sort_len:
    grouped = []
    unclassified = []
    grouped.append(item[0])
    for i in range(len(item)-1):
        if (item[0] in item[i+1]) or (item[0][:6] in item[i+1]):
            grouped.append(item[i+1])
        else:
            unclassified.append(item[i+1])
    done.append(grouped)
    if unclassified != []:
        unclassified_group.append(unclassified)

In [19]:
### FINAL GROUPING
for item in unclassified_group:
    if len(item) != 1:
        temp_grouped = []
        temp_unclassified = []
        temp_grouped.append(item[0])
        for i in range(len(item)-1):
            if (item[0] in item[i+1]) or (item[0][:len(item[0])//2] in item[i+1]):
                temp_grouped.append(item[i+1])
            else:
                temp_unclassified.append(item[i+1])
        done.append(temp_grouped)
        #print(temp_unclassified)
        if temp_unclassified != [] and len(temp_unclassified) == 1:
            unique.append(temp_unclassified[0])
    else:
        unique.append(item[0])

In [20]:
### DICTIONARY OF STORE NAME
dict_done = {}
for i in range(len(done)):
    dict_done[i] = done[i]
    
unique_ = []
for u in unique:
    unique_.append([u])
dict_unique = {i: unique_[j] for i,j in zip([ i for i in range(len(dict_done), len(unique)+len(dict_done))], [j for j in range(len(unique))])}

dict_fb = dict(dict_done)
dict_fb.update(dict_unique)

In [21]:
### MAPPING SAME STORE NAME WITH MAP
store_id = []
for store in list(df_new['stem_store']):
    for key, v in dict_fb.items():
        if store in v:
            store_id.append(key)
            break
        elif (key == list(dict_fb.keys())[-1]):
            store_id.append('9999')
df_new['store_id'] = store_id

In [22]:
### Load map.csv
df_table_1 = read_csv('Main_Store.csv', encoding='utf8',index_col=0)
map_id = list(df_table_1.index)
map_name = list(df_table_1['main_store'])

In [23]:
### Matching
fb_match_id = []

for fb_key, fb_v in dict_fb.items():
    fb_v[0] = fb_v[0].lower()
    fb_v[0] = fb_v[0].replace('の','之')
    for m_id, m_name in zip(map_id, map_name):
        m_name = m_name.lower()
        m_name = m_name.replace('の','之')
        if len(fb_v) == 1 and (fb_v[0] == m_name):
            fb_match_id.append(m_id)
            break
        elif (fb_v[0] == m_name):
            fb_match_id.append(m_id)
            break
        elif (len(fb_v) >= 1) and (fb_v[0] in m_name):
            fb_match_id.append(m_id)
            break
        elif (len(fb_v[0]) >= 6) and (fb_v[0][:5] == m_name[:5]):
            fb_match_id.append(m_id)
            #print(m_name[:6])
            break
        elif (len(fb_v[0]) >= 5) and (len(fb_v[0]) <= 7) and (fb_v[0][:4] == m_name[:4]):
            fb_match_id.append(m_id)
            #print(m_name)
            break       
        elif (m_id == 335):
            fb_match_id.append('99999')
            break           

In [24]:
### CREATE STORE-ID
'''
IF the store exsits in Table_1 (Main_Store), store_id matches together;
Otherwise, the store_id is 10000+number, which means this store only exists in FB's Post.
'''
table_foreign_key = []

for store in list(df_new['stem_store']):
    for (fb_k, fb_v), new in zip(dict_fb.items(), fb_match_id):
        if store in fb_v and (new != '99999'):
            table_foreign_key.append(new)
            break
        elif (store in fb_v) and (new == '99999'):
            table_foreign_key.append(fb_k + 10000)
            break
        elif (fb_k == list(dict_fb.keys())[-1]) and (store not in fb_v):
            table_foreign_key.append('99999')
            break

In [25]:
### Create post_id 
post_id = []
for i in range(1,len(table_foreign_key)+1):
    if len(str(i)) == 1:
        id_ = 'T3' + '0000' + str(i)
    elif len(str(i)) == 2:
        id_ = 'T3' + '000' + str(i)
    elif len(str(i)) == 3:
        id_ = 'T3' + '00' + str(i)
    elif len(str(i)) == 4:
        id_ = 'T3' + '0' + str(i)
    elif len(str(i)) == 6:
        id_ = 'T3' + str(i)
    post_id.append(id_)   
df_new['post_id'] = post_id

In [26]:
df_new['store_id'] = table_foreign_key 
df_t3 = df_new[['post_id','store_id','stores', 'create_on', 'ramen_name', 'fb_review']]
#df_t3.to_csv('Post.csv', index=False)